<a href="https://colab.research.google.com/github/jcmachicao/documentos_LLM/blob/main/cur_DocsT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.6/328.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


In [8]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.7 MB/s eta 0:00:00


In [9]:
import openai
import requests
import io
from PyPDF2 import PdfReader

In [21]:
from google.colab import userdata
OAI_APIKEY = userdata.get('OAIK_JCMV')
openai.api_key = OAI_APIKEY
from openai import OpenAI
client = OpenAI(api_key=OAI_APIKEY)

In [6]:
url_texto = 'https://biblioteca.spda.org.pe/biblioteca/catalogo/_data/20210604182730_informe-tecnico-legal-pucusana-spda.pdf'

In [12]:
def obtener_texto_pdf_desde_url(url, pag_ini, pag_fin):
    response = requests.get(url) # Descargar el PDF
    if response.status_code != 200:
        return "Error al descargar el PDF"

    pdf_file = io.BytesIO(response.content) # Crear un objeto de tipo archivo en memoria
    pdf_reader = PdfReader(pdf_file)     # Crear un objeto PdfReader
    texto = "" # Extraer texto de todas las páginas
    for pagina in pdf_reader.pages[pag_ini:pag_fin]:
        texto += pagina.extract_text()
    return texto

texto_pdf = obtener_texto_pdf_desde_url(url_texto, 1, 40)
print(texto_pdf[0:1000]) # Esto solo es la muestra del inicio del texto

Edición: 
Sociedad Peruana de Derecho Ambiental
Investigación y textos:
José Bringas, Fabio Castagnino, Percy Grandez,  
Ricardo Bandín y Bruno Monteferri.
Corrección de estilo: 
Milagros Bustamante
Diagramación:
Arantxa Gonzalez
Foto de portada:
Andina / Difusión
Se permite el uso de esta publicación siempre y cuando se cite 
correctamente su autoría.
Sociedad Peruana de Derecho Ambiental
Presidente: Jorge CaillauxDirectora ejecutiva: Isabel Calle Director de Gobernanza Marina: Bruno Monteferri Av. Prolongación Arenales 437, San Isidro, LimaTeléfono: (+51) 612-4700www.spda.org.pe
Gobernanza Marina es una iniciativa de la SPDA que busca 
mejorar la gobernanza para la gestión de las pesquerías y la conservación de ecosistemas marinos en el Perú.
Esta publicación es posible en el marco del proyecto “Hacia 
una gestión sostenible de las pesquerías a través de mejores regulaciones, aplicación de la ley y la transparencia en el Perú”, financiado por The Walton Family Foundation.Cita sugerid

In [28]:
def extractos_importantes(texto):
    requerimientos = f'''
    Lee el siguiente texto y realiza estas tareas:
    1. Extrae los 5 puntos clave más importantes.
    2. Identifica los rubros temáticos universales o conceptuales aludidos
    3. Identifica las controversias que alude el texto.

    Texto:
    {texto_pdf[0:2000]}
    '''

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {'role': 'system', 'content': 'Eres un experto en análisis de textos complejos.'},
            {"role": "user", "content": requerimientos},
            ],
        temperature=0.7,
        max_tokens=350,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
      )
    return response

In [33]:
# Ejemplo de uso
resumen_y_puntos = extractos_importantes(texto_pdf)

In [44]:
print(resumen_y_puntos.choices[0].message.content)

1. Puntos clave más importantes:
- La Sociedad Peruana de Derecho Ambiental está involucrada en la iniciativa de Gobernanza Marina para mejorar la gestión de pesquerías y conservación de ecosistemas marinos en Perú.
- Se menciona un proyecto financiado por The Walton Family Foundation que apunta a una gestión sostenible de las pesquerías en Perú.
- Se hace referencia a un informe técnico-legal sobre la problemática ambiental de la bahía de Pucusana, elaborado por la SPDA.
- Se presentan siglas y acrónimos relacionados con entidades y conceptos ambientales relevantes en el contexto peruano.
- Se incluyen detalles de contacto y cita sugerida para la Sociedad Peruana de Derecho Ambiental.

2. Rubros temáticos universales o conceptuales aludidos:
- Derecho ambiental y gobernanza para la gestión sostenible de recursos naturales.
- Conservación de ecosistemas marinos y pesquerías.
- Financiamiento de proyectos ambientales por organizaciones externas.
- Problemáticas ambientales específicas, 